# **DECISION TREE**

**ID3 ALGORITHM**

In [4]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np
from math import log2

iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = pd.Series(iris.target, name="target")
data = X.copy()
data["target"] = y

def entropy(column):
    values, counts = np.unique(column, return_counts=True)
    return -np.sum((counts / np.sum(counts)) * np.log2(counts / np.sum(counts)))

def info_gain(data, feature, target):
    total_entropy = entropy(data[target])
    thresholds = np.unique(data[feature])
    best_gain = 0
    best_thresh = None
    for t in thresholds:
        left = data[data[feature] <= t]
        right = data[data[feature] > t]
        if len(left) == 0 or len(right) == 0:
            continue
        p = len(left) / len(data)
        gain = total_entropy - (p * entropy(left[target]) + (1 - p) * entropy(right[target]))
        if gain > best_gain:
            best_gain = gain
            best_thresh = t
    return best_gain, best_thresh

def ID3(data, features, target, depth=0, max_depth=3):
    labels = data[target]
    if len(np.unique(labels)) == 1 or len(features) == 0 or depth == max_depth:
        return np.bincount(labels).argmax()
    gains = {}
    thresholds = {}
    for feature in features:
        gain, thresh = info_gain(data, feature, target)
        gains[feature] = gain
        thresholds[feature] = thresh
    best_feature = max(gains, key=gains.get)
    best_threshold = thresholds[best_feature]
    if best_threshold is None:
        return np.bincount(labels).argmax()
    tree = {f"{best_feature} <= {best_threshold:.2f}": {}}
    left = data[data[best_feature] <= best_threshold]
    right = data[data[best_feature] > best_threshold]
    tree[f"{best_feature} <= {best_threshold:.2f}"]["yes"] = ID3(left, features, target, depth+1, max_depth)
    tree[f"{best_feature} <= {best_threshold:.2f}"]["no"] = ID3(right, features, target, depth+1, max_depth)
    return tree

def print_tree(tree, indent=""):
    if isinstance(tree, dict):
        for key, value in tree.items():
            print(f"{indent}{key}?")
            for branch in value:
                print(f"{indent}→ {branch}:")
                print_tree(value[branch], indent + "    ")
    else:
        print(f"{indent}Predict → {tree}")
features = X.columns.tolist()
tree = ID3(data, features, "target", max_depth=3)
print_tree(tree)


petal length (cm) <= 1.90?
→ yes:
    Predict → 0
→ no:
    petal width (cm) <= 1.70?
    → yes:
        petal length (cm) <= 4.90?
        → yes:
            Predict → 1
        → no:
            Predict → 2
    → no:
        petal length (cm) <= 4.80?
        → yes:
            Predict → 2
        → no:
            Predict → 2


**C4.5 ALGORITHM**

In [2]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np
from math import log2

iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = pd.Series(iris.target, name="target")
data = X.copy()
data["target"] = y

def entropy(column):
    values, counts = np.unique(column, return_counts=True)
    return -np.sum((counts / np.sum(counts)) * np.log2(counts / np.sum(counts)))

def split_info(subsets, total_size):
    return -np.sum([(len(subset)/total_size) * log2(len(subset)/total_size) for subset in subsets if len(subset) > 0])

def gain_ratio(data, feature, target):
    total_entropy = entropy(data[target])
    thresholds = np.unique(data[feature])
    best_gain_ratio = 0
    best_thresh = None
    for t in thresholds:
        left = data[data[feature] <= t]
        right = data[data[feature] > t]
        if len(left) == 0 or len(right) == 0:
            continue
        p = len(left) / len(data)
        gain = total_entropy - (p * entropy(left[target]) + (1 - p) * entropy(right[target]))
        si = split_info([left, right], len(data))
        if si == 0:
            continue
        ratio = gain / si
        if ratio > best_gain_ratio:
            best_gain_ratio = ratio
            best_thresh = t
    return best_gain_ratio, best_thresh

def C45(data, features, target, depth=0, max_depth=3):
    labels = data[target]
    if len(np.unique(labels)) == 1 or len(features) == 0 or depth == max_depth:
        return np.bincount(labels).argmax()
    best_feature = None
    best_threshold = None
    best_ratio = -1
    for feature in features:
        ratio, thresh = gain_ratio(data, feature, target)
        if ratio > best_ratio:
            best_ratio = ratio
            best_feature = feature
            best_threshold = thresh
    if best_feature is None or best_threshold is None:
        return np.bincount(labels).argmax()
    tree = {f"{best_feature} <= {best_threshold:.2f}": {}}
    left = data[data[best_feature] <= best_threshold]
    right = data[data[best_feature] > best_threshold]
    tree[f"{best_feature} <= {best_threshold:.2f}"]["yes"] = C45(left, features, target, depth+1, max_depth)
    tree[f"{best_feature} <= {best_threshold:.2f}"]["no"] = C45(right, features, target, depth+1, max_depth)
    return tree

def print_tree(tree, indent=""):
    if isinstance(tree, dict):
        for key, value in tree.items():
            print(f"{indent}{key}?")
            for branch in value:
                print(f"{indent}→ {branch}:")
                print_tree(value[branch], indent + "    ")
    else:
        print(f"{indent}Predict → {tree}")

features = X.columns.tolist()
tree = C45(data, features, "target", max_depth=3)
print_tree(tree)


petal length (cm) <= 1.90?
→ yes:
    Predict → 0
→ no:
    petal width (cm) <= 1.70?
    → yes:
        petal length (cm) <= 5.10?
        → yes:
            Predict → 1
        → no:
            Predict → 2
    → no:
        petal length (cm) <= 4.80?
        → yes:
            Predict → 2
        → no:
            Predict → 2


**CART ALGORITHM**

In [3]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np

iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = pd.Series(iris.target, name="target")
data = X.copy()
data["target"] = y

def gini(column):
    values, counts = np.unique(column, return_counts=True)
    prob_sq = (counts / np.sum(counts)) ** 2
    return 1 - np.sum(prob_sq)

def gini_index(data, feature, target):
    thresholds = np.unique(data[feature])
    best_gini = float('inf')
    best_thresh = None
    for t in thresholds:
        left = data[data[feature] <= t]
        right = data[data[feature] > t]
        if len(left) == 0 or len(right) == 0:
            continue
        p_left = len(left) / len(data)
        p_right = len(right) / len(data)
        g = p_left * gini(left[target]) + p_right * gini(right[target])
        if g < best_gini:
            best_gini = g
            best_thresh = t
    return best_gini, best_thresh

def CART(data, features, target, depth=0, max_depth=3):
    labels = data[target]
    if len(np.unique(labels)) == 1 or len(features) == 0 or depth == max_depth:
        return np.bincount(labels).argmax()
    best_feature = None
    best_threshold = None
    best_gini = float('inf')
    for feature in features:
        gini_val, thresh = gini_index(data, feature, target)
        if gini_val < best_gini:
            best_gini = gini_val
            best_feature = feature
            best_threshold = thresh
    if best_feature is None or best_threshold is None:
        return np.bincount(labels).argmax()
    tree = {f"{best_feature} <= {best_threshold:.2f}": {}}
    left = data[data[best_feature] <= best_threshold]
    right = data[data[best_feature] > best_threshold]
    tree[f"{best_feature} <= {best_threshold:.2f}"]["yes"] = CART(left, features, target, depth+1, max_depth)
    tree[f"{best_feature} <= {best_threshold:.2f}"]["no"] = CART(right, features, target, depth+1, max_depth)
    return tree

def print_tree(tree, indent=""):
    if isinstance(tree, dict):
        for key, value in tree.items():
            print(f"{indent}{key}?")
            for branch in value:
                print(f"{indent}→ {branch}:")
                print_tree(value[branch], indent + "    ")
    else:
        print(f"{indent}Predict → {tree}")

features = X.columns.tolist()
tree = CART(data, features, "target", max_depth=3)
print_tree(tree)


petal length (cm) <= 1.90?
→ yes:
    Predict → 0
→ no:
    petal width (cm) <= 1.70?
    → yes:
        petal length (cm) <= 4.90?
        → yes:
            Predict → 1
        → no:
            Predict → 2
    → no:
        petal length (cm) <= 4.80?
        → yes:
            Predict → 2
        → no:
            Predict → 2
